# 🔍 Project Summary

This project organizes service-related data using Neo4j and py2neo to show how different people and resources are connected in completing service tasks (like home spa treatments or repairs).

Each task involves:

- A customer,

- A driver,

- A vehicle,

- One or more technicians,

- And details like time, area, and cost.

We started by creating basic nodes like employees and departments, then gradually connected each task to the involved people and resources.

By looping through multiple tasks, we built a full network graph that clearly shows who does what, when, and how — like building a smart map of every service job.

In [265]:
import json
from py2neo import Graph, Node, Relationship, NodeMatcher

In [266]:
#Make Connection
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

In [267]:
# Load employee data from the JSON file
with open("data/field_service.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [268]:
print(type(data))

<class 'list'>


In [269]:
print("Top-level keys in first item:")
print(data[5].keys())

Top-level keys in first item:
dict_keys(['service_task', 'customer', 'technicians', 'driver', 'vehicle', 'planned_date', 'time_slot', 'invoice'])


In [270]:
print(data[5]["service_task"])

{'id': 8293, 'name': 'Classic Manicure', 'area': 'Jumeriah Park', 'servis_name': 'Field Service', 'standard_name': 'Classic Manicure', 'similarity': '-1'}


In [271]:
print(data[5]["service_task"]["area"])

Jumeriah Park


# 1. Create First Task 

## 1.1 Create Task Node

In [272]:
task_node = Node("Task",
            id = data[5]["service_task"]["id"],
            name = data[5]["service_task"]["standard_name"],
            area = data[5]["service_task"]["area"],
            project = data[5]["service_task"]["servis_name"],
            planned_date = data[5]["planned_date"]["date"],
            start_time = data[5]["time_slot"]["start"],
            end_time = data[5]["time_slot"]["end"],
            amount = data[5]["invoice"]["amount"],
            order_reference = data[5]["invoice"]["order_reference"])
graph.merge(task_node, "Task", "id")

![Task](data/pics/task.png)

## 1.2 Create Customer Node

In [273]:
customer_node = Node("Customer",
                id = data[5]["customer"]["id"],
                name = data[5]["customer"]["name"],
                gender = data[5]["customer"]["gender"],
                contact = data[5]["customer"]["contact"],
                address = data[5]["customer"]["address"],
                email = data[5]["customer"]["email"],
                map_url = data[5]["customer"]["map_url"])
graph.merge(customer_node, "Customer", "id")

![Task](data/pics/customer.png)

## 1.3 Create Driver Node

In [274]:
driver_node = Node("Employee",
                   id=data[5]["driver"]["id"],
                   name=data[5]["driver"]["name"])
graph.merge(driver_node, "Employee", "id")       

![Task](data/pics/driver.png)

## 1.4 Create Vehicle

In [275]:
vehicle_node = Node("Vehicle",
               id = data[5]["vehicle"]["id"],
               name = data[5]["vehicle"]["name"],
               license_plate = data[5]["vehicle"]["license_plate"])
graph.merge(vehicle_node, "Vehicle", "id")       

![Task](data/pics/vehicle.png)

## 1.5 Create Technicians Node

In [252]:
print(data[5]["technicians"].keys())

dict_keys(['selected_technician', 'preferred_technician', 'non_preferred_technician'])


In [253]:
print(type(data[5]["technicians"]["selected_technician"]))

<class 'list'>


In [254]:
print(data[5]["technicians"]["selected_technician"][0])

{'id': 434, 'resource_id': 357, 'name': 'Rollyn Quitor Salvacion'}


In [276]:
selected_techs = data[5]["technicians"].get("selected_technician", [])
technician_node = []
for tech in selected_techs:
    tech_node = Node("Employee",
                           id=tech["id"],
                           resource_id=tech["resource_id"],
                           name=tech["name"])
    graph.merge(tech_node, "Employee", "id")  # using id as unique key
    technician_node.append(tech_node)

![Task](data/pics/technician.png)

## 1.5 Create Relations

In [279]:
# Link Task to Customer
graph.merge(Relationship(task_node, "HAS_CUSTOMER", customer_node))

# Link Task to Driver
graph.merge(Relationship(task_node, "HAS_DRIVER", driver_node))

# Link Task to Vehicle
graph.merge(Relationship(task_node, "USES_VEHICLE", vehicle_node))

for tech in technician_node:
    graph.merge(Relationship(task_node, "SELECTED_TECHNICIAN", tech))

![Task](data/pics/relation.png)

# 2. Create Graph & Relations For First 10 Tasks

In [280]:
for item in data[:10]:
    # 1. Create Task node
    task_data = item.get("service_task", {})
    planned_date = item.get("planned_date", {}).get("date")
    time_slot = item.get("time_slot", {})
    invoice = item.get("invoice", {})

    task = Node("Task",
                id=task_data.get("id"),
                name=task_data.get("standard_name"),
                area=task_data.get("area"),
                project=task_data.get("servis_name"),
                planned_date=planned_date,
                start_time=time_slot.get("start"),
                end_time=time_slot.get("end"),
                amount=invoice.get("amount"),
                order_reference=invoice.get("order_reference"))
    graph.merge(task, "Task", "id")

    # 2. Create Customer node (only if ID is present)
    customer_data = item.get("customer", {})
    if customer_data.get("id") is not None:
        customer = Node("Customer",
                        id=customer_data.get("id"),
                        name=customer_data.get("name"),
                        gender=customer_data.get("gender"),
                        contact=customer_data.get("contact"),
                        address=customer_data.get("address"),
                        email=customer_data.get("email"),
                        map_url=customer_data.get("map_url"))
        graph.merge(customer, "Customer", "id")
        graph.merge(Relationship(task, "HAS_CUSTOMER", customer))

    # 3. Create Driver node
    driver_data = item.get("driver", {})
    if driver_data.get("id") is not None:
        driver = Node("Employee",
                      id=driver_data.get("id"),
                      name=driver_data.get("name"))
        graph.merge(driver, "Employee", "id")
        graph.merge(Relationship(task, "HAS_DRIVER", driver))

        # 4. Create Vehicle and relationship
        vehicle_data = item.get("vehicle", {})
        if vehicle_data.get("id") is not None:
            vehicle = Node("Vehicle",
                           id=vehicle_data.get("id"),
                           name=vehicle_data.get("name"),
                           license_plate=vehicle_data.get("license_plate"))
            graph.merge(vehicle, "Vehicle", "id")
            graph.merge(Relationship(driver, "DRIVES", vehicle))
            graph.merge(Relationship(task, "USES_VEHICLE", vehicle))

    # 5. Create Technician nodes and relationships
    selected_techs = item.get("technicians", {}).get("selected_technician", [])
    for tech in selected_techs:
        if tech.get("id") is not None:
            technician_node = Node("Employee",
                                   id=tech.get("id"),
                                   resource_id=tech.get("resource_id"),
                                   name=tech.get("name"))
            graph.merge(technician_node, "Employee", "id")
            graph.merge(Relationship(task, "SELECTED_TECHNICIAN", technician_node))

![Task](data/pics/tenTask.png)